# Tutorial 2: SMM Basics

This tutorial introduces the **Simulated Method of Moments (SMM)** and demonstrates how to use `momentest` for SMM estimation.

## What You'll Learn

1. What SMM is and when to use it (vs GMM)
2. How to define simulation and moment functions
3. The role of Common Random Numbers (CRN)
4. How to estimate parameters using `smm_estimate()`

## Prerequisites

- Completed Tutorial 1 (GMM Basics)
- Understanding of simulation methods

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts

# Import momentest
from momentest import (
    smm_estimate,
    consumption_savings,
    table_estimates,
    confidence_interval,
    plot_moment_comparison,
    plot_objective_landscape,
    load_econ381,
)

np.random.seed(42)

## 1. What is SMM?

**Simulated Method of Moments (SMM)** is like GMM, but instead of computing moments analytically from data, we **simulate** moments from a model.

### GMM vs SMM

| Aspect | GMM | SMM |
|--------|-----|-----|
| Moments | Computed from data | Simulated from model |
| Use case | Analytical moment conditions | Complex structural models |
| Computation | Fast | Slower (requires simulation) |

### When to Use SMM

- **Complex models**: When moments can't be computed analytically
- **Structural estimation**: Dynamic models, discrete choice, etc.
- **Latent variables**: When the model involves unobserved heterogeneity

### The SMM Objective

$$\hat{\theta} = \arg\min_\theta (\bar{m}(\theta) - m_{data})' W (\bar{m}(\theta) - m_{data})$$

where $\bar{m}(\theta) = \frac{1}{S}\sum_{s=1}^S m_s(\theta)$ is the average of simulated moments.

## 2. Common Random Numbers (CRN)

A key technique in SMM is **Common Random Numbers (CRN)**:

- Pre-draw random shocks **once** before optimization
- Use the **same shocks** for all parameter evaluations
- This makes the objective function **smooth** (differentiable)

Without CRN, the objective would be noisy and optimization would fail!

```python
# CRN: Same shocks for all θ
shocks = np.random.randn(n_sim, shock_dim)  # Pre-drawn once

def objective(theta):
    sim_data = simulate(theta, shocks)  # Same shocks!
    return compute_criterion(sim_data)
```

## 3. Example: Truncated Normal Distribution

Let's estimate a **truncated normal distribution** using SMM. This is a classic example from econometrics courses.

### The Model

- Latent variable: $X^* \sim N(\mu, \sigma^2)$
- Observed: $X = X^*$ if $a \leq X^* \leq b$, else truncated
- Parameters: $\theta = (\mu, \sigma)$

### Load Data

In [ ]:
# Load the Econ 381 test scores dataset
dataset = load_econ381()
data = dataset['data']
cut_lb, cut_ub = dataset['bounds']  # Truncation bounds: [0, 450]

print(f"Dataset: {dataset['n']} test scores")
print(f"Bounds: [{cut_lb}, {cut_ub}]")
print(f"Sample mean: {data.mean():.2f}")
print(f"Sample variance: {data.var():.2f}")
print(f"\nMLE estimates (target): μ={dataset['mle_params']['mu']}, σ={dataset['mle_params']['sigma']}")

In [ ]:
# Visualize the data
plt.figure(figsize=(10, 5))
plt.hist(data, bins=30, density=True, alpha=0.7, edgecolor='black')
plt.xlabel('Test Score')
plt.ylabel('Density')
plt.title('Econ 381 Test Scores (Truncated Normal)')
plt.axvline(cut_lb, color='red', linestyle='--', label=f'Lower bound ({cut_lb})')
plt.axvline(cut_ub, color='red', linestyle='--', label=f'Upper bound ({cut_ub})')
plt.legend()
plt.show()

## 4. Define Simulation and Moment Functions

For SMM, we need two functions:

1. **`sim_func(theta, shocks)`**: Simulates data given parameters and random shocks
2. **`moment_func(sim_data)`**: Computes moments from simulated data

In [ ]:
def trunc_norm_draws(unif_vals, mu, sigma, cut_lb, cut_ub):
    """
    Draw from truncated normal using inverse CDF method.
    
    Args:
        unif_vals: Uniform(0,1) random values
        mu, sigma: Normal distribution parameters
        cut_lb, cut_ub: Truncation bounds
    
    Returns:
        Truncated normal draws
    """
    sigma = max(sigma, 1e-6)
    
    # CDF at bounds
    cut_lb_cdf = sts.norm.cdf(cut_lb, loc=mu, scale=sigma)
    cut_ub_cdf = sts.norm.cdf(cut_ub, loc=mu, scale=sigma)
    cdf_range = cut_ub_cdf - cut_lb_cdf
    
    if cdf_range < 1e-10:
        return np.full_like(unif_vals, (cut_lb + cut_ub) / 2)
    
    # Scale uniform to truncated range and invert
    unif_scaled = unif_vals * cdf_range + cut_lb_cdf
    unif_scaled = np.clip(unif_scaled, cut_lb_cdf + 1e-10, cut_ub_cdf - 1e-10)
    draws = sts.norm.ppf(unif_scaled, loc=mu, scale=sigma)
    
    return np.clip(draws, cut_lb, cut_ub)

In [ ]:
# Number of observations to simulate per draw
N = len(data)

def sim_func(theta, shocks):
    """
    Simulate truncated normal data.
    
    Args:
        theta: [mu, sigma] parameters
        shocks: Uniform(0,1) values of shape (n_sim, N)
    
    Returns:
        Simulated data of shape (n_sim, N)
    """
    mu, sigma = theta
    sigma = max(sigma, 1.0)  # Ensure positive
    return trunc_norm_draws(shocks, mu, sigma, cut_lb, cut_ub)


def moment_func(sim_data):
    """
    Compute moments (mean and variance) for each simulation.
    
    Args:
        sim_data: Simulated data of shape (n_sim, N)
    
    Returns:
        Moments of shape (n_sim, 2) - [mean, variance] per simulation
    """
    sim_means = np.mean(sim_data, axis=1)
    sim_vars = np.var(sim_data, axis=1)
    return np.column_stack([sim_means, sim_vars])

## 5. SMM Estimation with momentest

Now let's estimate using `smm_estimate()`. This is the simple, high-level API:

In [ ]:
# Target moments from data
data_mean = data.mean()
data_var = data.var()
data_moments = [data_mean, data_var]

print(f"Target moments: mean={data_mean:.2f}, var={data_var:.2f}")

In [ ]:
# SMM estimation - just a few lines!
result = smm_estimate(
    sim_func=sim_func,           # Our simulation function
    moment_func=moment_func,     # Our moment function
    data_moments=data_moments,   # Target moments from data
    bounds=[(0, 1000), (1, 500)], # Parameter bounds: (mu, sigma)
    n_sim=300,                   # Number of simulations
    shock_dim=N,                 # Dimension of shocks (N observations)
    seed=42,                     # Random seed for CRN
    weighting="optimal",        # Two-step optimal weighting
)

print(result)

In [ ]:
# Compare to MLE
mle_mu = dataset['mle_params']['mu']
mle_sigma = dataset['mle_params']['sigma']

print("\n" + "="*60)
print("COMPARISON")
print("="*60)
print(f"{'Method':<15} {'μ':>12} {'σ':>12}")
print("-"*40)
print(f"{'MLE':<15} {mle_mu:>12.2f} {mle_sigma:>12.2f}")
print(f"{'SMM':<15} {result.theta[0]:>12.2f} {result.theta[1]:>12.2f}")
print("="*60)
print("\nSMM recovers parameters close to MLE!")

## 6. Understanding the Results

### Parameter Estimates

In [ ]:
# Formatted table
ci_lower, ci_upper = confidence_interval(result.theta, result.se)

print(table_estimates(
    theta=result.theta,
    se=result.se,
    param_names=["μ (mean)", "σ (std dev)"],
    ci_lower=ci_lower,
    ci_upper=ci_upper,
))

### Moment Fit

In [ ]:
# Compare data vs simulated moments
print(f"Data moments:      mean={data_mean:.2f}, var={data_var:.2f}")
print(f"Simulated moments: mean={result.sim_moments[0]:.2f}, var={result.sim_moments[1]:.2f}")
print(f"\nObjective value: {result.objective:.2e}")

In [ ]:
# Visualize moment fit
fig = plot_moment_comparison(
    data_moments=np.array(data_moments),
    model_moments=result.sim_moments,
    moment_names=["Mean", "Variance"],
)
plt.suptitle("SMM Moment Fit", y=1.02)
plt.tight_layout()
plt.show()

## 7. Visualize the Fitted Distribution

In [ ]:
def trunc_norm_pdf(x, mu, sigma, cut_lb, cut_ub):
    """PDF of truncated normal."""
    sigma = max(sigma, 1e-6)
    prob_notcut = sts.norm.cdf(cut_ub, loc=mu, scale=sigma) - sts.norm.cdf(cut_lb, loc=mu, scale=sigma)
    if prob_notcut < 1e-10:
        return np.zeros_like(x)
    pdf = sts.norm.pdf(x, loc=mu, scale=sigma) / prob_notcut
    return pdf

# Plot
plt.figure(figsize=(10, 6))
plt.hist(data, bins=30, density=True, alpha=0.6, edgecolor='black', label='Data')

x_vals = np.linspace(cut_lb, cut_ub, 500)
plt.plot(x_vals, trunc_norm_pdf(x_vals, mle_mu, mle_sigma, cut_lb, cut_ub),
         'g-', linewidth=2.5, label=f'MLE: μ={mle_mu:.0f}, σ={mle_sigma:.0f}')
plt.plot(x_vals, trunc_norm_pdf(x_vals, result.theta[0], result.theta[1], cut_lb, cut_ub),
         'r--', linewidth=2, label=f'SMM: μ={result.theta[0]:.0f}, σ={result.theta[1]:.0f}')

plt.xlabel('Test Score')
plt.ylabel('Density')
plt.title('SMM Estimation: Truncated Normal')
plt.legend()
plt.xlim([0, 500])
plt.show()

## 8. The Role of CRN: A Demonstration

Let's see why CRN is important by comparing objectives with and without it:

In [ ]:
# Pre-draw shocks for CRN
np.random.seed(42)
fixed_shocks = np.random.uniform(0, 1, size=(300, N))

def objective_with_crn(theta):
    """Objective using CRN (same shocks)."""
    sim_data = sim_func(theta, fixed_shocks)
    moments = moment_func(sim_data)
    m_bar = moments.mean(axis=0)
    g = m_bar - np.array(data_moments)
    return float(g @ g)

def objective_without_crn(theta):
    """Objective without CRN (new shocks each time)."""
    new_shocks = np.random.uniform(0, 1, size=(300, N))
    sim_data = sim_func(theta, new_shocks)
    moments = moment_func(sim_data)
    m_bar = moments.mean(axis=0)
    g = m_bar - np.array(data_moments)
    return float(g @ g)

# Evaluate along a path
mu_vals = np.linspace(500, 700, 50)
sigma_fixed = 200

obj_crn = [objective_with_crn([mu, sigma_fixed]) for mu in mu_vals]

np.random.seed(123)  # Different seed for non-CRN
obj_no_crn = [objective_without_crn([mu, sigma_fixed]) for mu in mu_vals]

# Plot
plt.figure(figsize=(10, 5))
plt.plot(mu_vals, obj_crn, 'b-', linewidth=2, label='With CRN (smooth)')
plt.plot(mu_vals, obj_no_crn, 'r-', linewidth=1, alpha=0.7, label='Without CRN (noisy)')
plt.xlabel('μ')
plt.ylabel('Objective')
plt.title('CRN Makes the Objective Smooth')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("With CRN: The objective is smooth and optimization works well.")
print("Without CRN: The objective is noisy and optimization may fail!")

## 9. Example: Consumption-Savings Model

Let's try a more structural example - the two-period consumption-savings model:

In [ ]:
# Generate data from consumption-savings DGP
dgp = consumption_savings(n=1000, seed=42, beta=0.95, gamma=2.0)
dgp.info()

In [ ]:
# This model has analytical moment conditions, so we can use GMM
# But let's demonstrate SMM approach

# For SMM, we'd simulate the model and match moments
# The DGP provides the moment function for GMM
# Here we show the true parameters we're trying to recover

print(f"True parameters: β={dgp.true_theta[0]}, γ={dgp.true_theta[1]}")
print(f"\nThis is an intermediate-level model.")
print("See Tutorial 6 for advanced structural models.")

## 10. Exercises

### Exercise 1: More Simulations
Try different values of `n_sim` (100, 500, 2000). How does it affect estimates and standard errors?

### Exercise 2: Different Moments
Instead of mean and variance, try using percentiles (25th, 50th, 75th) as moments.

### Exercise 3: Identity vs Optimal Weighting
Compare `weighting="identity"` vs `weighting="optimal"`. Which gives lower standard errors?

### Exercise 4: Seed Sensitivity
Run estimation with different seeds. How much do estimates vary?

In [ ]:
# Exercise 1 starter code
for n_sim in [100, 300, 1000]:
    result_ex = smm_estimate(
        sim_func=sim_func,
        moment_func=moment_func,
        data_moments=data_moments,
        bounds=[(0, 1000), (1, 500)],
        n_sim=n_sim,
        shock_dim=N,
        seed=42,
        weighting="optimal",
    )
    print(f"n_sim={n_sim:4d}: μ={result_ex.theta[0]:.1f}, σ={result_ex.theta[1]:.1f}, SE(μ)={result_ex.se[0]:.1f}")

## Summary

In this tutorial, you learned:

1. **SMM basics**: Matching simulated moments to data moments
2. **Two key functions**:
   - `sim_func(theta, shocks)` → simulated data
   - `moment_func(sim_data)` → moments
3. **CRN**: Pre-drawn shocks make the objective smooth
4. **Simple API**: `smm_estimate()` handles everything

### Key Takeaways

- SMM is powerful for complex structural models
- CRN is essential for smooth optimization
- More simulations → lower variance but slower
- Optimal weighting improves efficiency

### Next Steps

- **Tutorial 3**: Optimal weighting in depth
- **Tutorial 4**: Bootstrap inference
- **Tutorial 6**: Advanced structural models